In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Creating the train and test data

In [ ]:
train = pd.read_csv('/kaggle/input/mercedes-benz-greener-manufacturing/train.csv.zip',compression='zip')
test = pd.read_csv('/kaggle/input/mercedes-benz-greener-manufacturing/test.csv.zip',compression='zip')
test_id = test['ID']
train.head()

## Check variance of each column

In [ ]:
train.var()

In [ ]:
dtype_df = train.dtypes.reset_index()
dtype_df.columns = ["Count", "Column Type"]
category_df = dtype_df.loc[dtype_df['Column Type'] == 'object']
category_list = list(category_df['Count'])
category_list

In [ ]:
from sklearn import preprocessing
for f in category_list:
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(train[f].values))
    train[f] = lbl.transform(list(train[f].values))
train.head()

In [ ]:
for f in category_list:
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(test[f].values))
    test[f] = lbl.transform(list(test[f].values))
test.head()

In [ ]:
train.info()

**Step 3: Data Cleaning and Preparation**

In [ ]:
train.isnull().sum().any()  ## This means no null data

Now check for variance for each group

In [ ]:
var_Data = train.var().reset_index()
var_Data.columns =['Columns','Variance']
zero_var = var_Data.loc[var_Data.Variance == 0]
zeroVarList = list(zero_var['Columns'])
zeroVarList




In [ ]:
train = train.drop(zeroVarList, axis = 1)

In [ ]:
test = test.drop(['X11', 'X93', 'X107', 'X233', 'X235', 'X268', 'X289', 'X290', 'X293', 'X297', 'X330', 'X347'], axis = 1)

In [ ]:
train.info()

In [ ]:
val_count  = train['X0'].value_counts()
vals = len(val_count)
val_count.index

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
for i in category_list:
    val_count  = train[i].value_counts()
    vals = len(val_count)
    #val_count.index
    fig, ax = plt.subplots(figsize=(15, 5))
    plt.xlabel('Feature value')
    plt.ylabel('Occurences')
    plt.title('Feature Vs Occurance for ' + str(i))
    plt.bar(val_count.index, val_count.values)
    ax.set_xticks(range(len(val_count)))
    plt.show()

Based on above graphs we can see the feature X4 has a imbalanced distribution of values. We can remove X4 feature

In [ ]:
train = train.drop('X4',axis = 1)

In [ ]:
test = test.drop('X4',axis = 1)

In [ ]:
test.head()

In [ ]:
test.head()

## Using xGBoost for training

In [ ]:
import xgboost as xgb
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [ ]:
train['y']

In [ ]:
X = train.drop(['ID','y'],axis = 1)
y = train['y']
x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=4242)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

In [ ]:
test = test.drop(['ID'], axis = 1)
test.shape
test.head()

In [ ]:
d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_valid, label=y_valid)
d_test = xgb.DMatrix(test)

In [ ]:
params = {}
params['objective'] = 'reg:linear'
params['eta'] = 0.02
params['max_depth'] = 4

In [ ]:
clf = xgb.train(params, d_train, 1000)

In [ ]:
p_test = clf.predict(d_test)

In [ ]:
p_test

In [ ]:
sub = pd.DataFrame()
sub['ID'] = test_id
sub['y'] = p_test
sub.to_csv('xgb.csv', index=False)